In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [3]:
import os
import shutil
import numpy as np
import pandas as pd
import yaml
from sklearn.model_selection import KFold
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
BASE_DIR = '/content/drive/MyDrive/dataset'
IMG_DIRS = [os.path.join(BASE_DIR, split, "images") for split in ["train", "valid", "test"]]
LBL_DIRS = [os.path.join(BASE_DIR, split, "labels") for split in ["train", "valid", "test"]]

In [5]:
all_images = []
for img_dir in IMG_DIRS:
    imgs = [os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith(".jpg")]
    all_images.extend(imgs)

def get_label(img_path):
    return img_path.replace("/images/", "/labels/").replace('.jpg', '.txt')

all_labels = [get_label(img) for img in all_images]

In [6]:
KFOLDS = 5
EPOCHS = 100
BATCH = 16
IMG_SIZE = 640
ORIG_YAML = os.path.join(BASE_DIR, "data.yaml")
metrics_list = []

In [7]:
kf = KFold(n_splits=KFOLDS, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(all_images), 1):
    print(f"\n=== Fold {fold}/{KFOLDS} ===")

    train_txt = f"train_fold{fold}.txt"
    val_txt = f"val_fold{fold}.txt"
    with open(train_txt, 'w') as f:
        f.write('\n'.join(all_images[i] for i in train_idx))
    with open(val_txt, 'w') as f:
        f.write('\n'.join(all_images[i] for i in val_idx))

    with open(ORIG_YAML, 'r') as f:
        data = yaml.safe_load(f)
    data['train'] = train_txt
    data['val'] = val_txt
    yaml_this_fold = f"data_fold{fold}.yaml"
    with open(yaml_this_fold, 'w') as f:
        yaml.safe_dump(data, f)

    model = YOLO("yolov8s.pt")
    run = model.train(
        data=yaml_this_fold,
        epochs=EPOCHS,
        patience=10,
        batch=BATCH,
        imgsz=IMG_SIZE,
        lr0=0.001,
        cos_lr=True,
        workers=4,
        device=0,
        augment=True,
        verbose=False,
        project="runs",
        name=f"fold{fold}_train",
        exist_ok=True
    )

    run_dir = str(run) if isinstance(run, str) else str(run.save_dir)

    best_weights = os.path.join(run_dir, "weights", "best.pt")
    shutil.copy(best_weights, f"glove_detector_best_fold{fold}.pt")

    results_csv = os.path.join(run_dir, "results.csv")
    df = pd.read_csv(results_csv)
    final_metrics = df.iloc[-1].to_dict()

    metrics_list.append({
        'mAP50': final_metrics.get('metrics/mAP50(B)', 0),
        'precision': final_metrics.get('metrics/precision(B)', 0),
        'recall': final_metrics.get('metrics/recall(B)', 0),
    })


=== Fold 1/5 ===


Ultralytics 8.3.177 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data_fold1.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fold1_train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=True, profi

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

Model summary: 129 layers, 11,136,374 parameters, 11,136,358 gradients, 28.6 GFLOPs

Transferred 349/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.5±0.1 ms, read: 0.1±0.0 MB/s, size: 44.1 KB)


...

     62/100      4.87G     0.7338     0.4597      1.137         15        640: 100%|██████████| 86/86 [00:32<00:00,  2.68it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:03<00:00,  2.95it/s]

                   all        341        359      0.973      0.951      0.971      0.783
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 52, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



62 epochs completed in 0.645 hours.
Optimizer stripped from runs/fold5_train/weights/last.pt, 22.5MB
Optimizer stripped from runs/fold5_train/weights/best.pt, 22.5MB

Validating runs/fold5_train/weights/best.pt...
Ultralytics 8.3.177 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:08<00:00,  1.34it/s]


                   all        341        359      0.955       0.96      0.975      0.801
Speed: 0.4ms preprocess, 13.8ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to runs/fold5_train


In [8]:
all_map50 = [m['mAP50'] for m in metrics_list]
all_prec = [m['precision'] for m in metrics_list]
all_rec = [m['recall'] for m in metrics_list]

print("\n=== Final CV Results ===")
print(f"mAP50:     {np.mean(all_map50):.4f} ± {np.std(all_map50):.4f}")
print(f"Precision: {np.mean(all_prec):.4f} ± {np.std(all_prec):.4f}")
print(f"Recall:    {np.mean(all_rec):.4f} ± {np.std(all_rec):.4f}")


=== Final CV Results ===
mAP50:     0.9743 ± 0.0031
Precision: 0.9735 ± 0.0097
Recall:    0.9547 ± 0.0076
